In [8]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

#读取数据
minst = input_data.read_data_sets("./data", one_hot=True)
#x容纳图片的28*28的二维数组
x = tf.placeholder(tf.float32, [None,784]) #输入X-训练
w = tf.Variable(tf.zeros([784,10])) #10代表神经元的个数
b = tf.Variable(tf.zeros([10]))
y = tf.placeholder(tf.float32,[None,10]) #输出y-结果
# y = w*x+b 预测
y_new = tf.nn.softmax(tf.matmul(x,w)+b)

# 定义损失函数，交叉熵
loss = tf.reduce_mean(-tf.reduce_sum(y*tf.log(y_new), reduction_indices=[1]))
# 训练的目标，降低损失
train = tf.train.GradientDescentOptimizer(0.01).minimize(loss)
# 全局初始化
init = tf.global_variables_initializer()

# tensorflow是个框架，python控制执行，_代表参数
def main(_):
    with tf.Session() as sess: #会话过程
        sess.run(init) #全局初始化
        for i in range(1000):
            #抓取数据
            batch_xs,batch_ys = minst.train.next_batch(200)
            #开始训练
            sess.run(train,feed_dict={x:batch_xs,y:batch_ys})
            #输出结果中最优的取出来
            last = tf.equal(tf.argmax(y,1),tf.argmax(y_new,1))
            accuracy = tf.reduce_mean(tf.cast(last,"float"))
            if i%10 == 0:#训练10次，测试效率
                print("  ",sess.run(accuracy,feed_dict={x:minst.test.images,y:minst.test.labels}))
            
if __name__ == "__main__":
    tf.app.run() #执行过程

Extracting ./data\train-images-idx3-ubyte.gz
Extracting ./data\train-labels-idx1-ubyte.gz
Extracting ./data\t10k-images-idx3-ubyte.gz
Extracting ./data\t10k-labels-idx1-ubyte.gz
   0.27
   0.6395
   0.7259
   0.7251
   0.7347
   0.7541
   0.7623
   0.7661
   0.7792
   0.7893
   0.793
   0.7975
   0.8099
   0.8031
   0.8039
   0.8028
   0.8096
   0.8116
   0.8143
   0.8143
   0.8175
   0.8181
   0.8192
   0.8227
   0.8255
   0.8279
   0.8283
   0.828
   0.8317
   0.8317
   0.8324
   0.8333
   0.8349
   0.8381
   0.839
   0.8392
   0.8404
   0.8412
   0.8416
   0.8421
   0.8435
   0.8436
   0.8445
   0.8444
   0.8456
   0.8456
   0.8456
   0.8474
   0.8487
   0.8493
   0.8503
   0.8515
   0.8529
   0.8537
   0.8528
   0.8551
   0.8564
   0.8568
   0.857
   0.8572
   0.8578
   0.8578
   0.858
   0.8574
   0.8581
   0.8594
   0.8601
   0.8608
   0.8619
   0.8625
   0.8615
   0.8635
   0.8629
   0.8638
   0.8654
   0.8642
   0.8642
   0.8637
   0.8643
   0.8642
   0.865
   0.8645
   0.8652


SystemExit: 

e:\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2918: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [14]:
from keras.datasets import mnist
import keras #深度学习框架
import keras.models #模型
from keras.models import Sequential #神经网络
from keras.layers import Dense,Dropout,Flatten #处理神经网络层
from keras.layers import Conv2D,MaxPooling2D #处理平面数据
from keras import backend as K #处理结束
import numpy as np

batch_size = 128 #批量操作大小
num_classes = 10 #识别的结果
epochs = 12 #训练12次
# 图片大小
img_rows,img_cols = 28,28
# 载入数据
# (x_train,y_train),(x_test,y_test) = mnist.load_data()
path='./MNIST_data/mnist.npz'
f = np.load(path)
x_train, y_train = f['x_train'], f['y_train']
x_test, y_test = f['x_test'], f['y_test']
f.close()
if K.image_data_format() == "channels_first":
    x_train = x_train.reshape(x_train.shape[0],1,img_rows,img_cols) #形状调整
    x_test = x_train.reshape(x_test.shape[0],1,img_rows,img_cols)
    input_shape=(1,img_rows,img_cols) #用于训练数据的形状1*28*28
else:
    x_train = x_train.reshape(x_train.shape[0],img_rows,img_cols,1) #形状调整
    x_test = x_test.reshape(x_test.shape[0],img_rows,img_cols,1)
    input_shape=(img_rows,img_cols,1) #用于训练数据的形状1*28*28
x_train = x_train.astype("float32")
x_test = x_test.astype("float32")
x_train /= 255 #xtrain矩阵的浮点类型，0-255 256，颜色改成203，0.78
x_test /= 255
print(x_train.shape[0],"样本数量")
print(x_test.shape[0],"样本数量")
print(x_train.shape,x_test.shape)
# 结果数据，0-9，10类，分类输出10个结果
y_train = keras.utils.np_utils.to_categorical(y_train,num_classes)
y_test = keras.utils.np_utils.to_categorical(y_test,num_classes)

model = Sequential() #新建一个神经网络
model.add(Conv2D(32,activation="relu",input_shape=input_shape,nb_row=3,nb_col=3))
model.add(Conv2D(64,activation="relu",nb_row=3,nb_col=3))
model.add(MaxPooling2D(pool_size=(2,2))) #搜索最优结果
model.add(Dropout(0.35)) #输出系数
model.add(Flatten()) #平整处理数据
model.add(Dense(128,activation="relu"))
model.add(Dropout(0.5)) #输出系数
model.add(Dense(num_classes,activation="softmax"))
model.compile(loss=keras.metrics.categorical_crossentropy, #训练的损失函数
              optimizer=keras.optimizers.Adadelta(),#优化
              metrics=['accuracy']) #精确
model.fit(x_train,y_train,
          batch_size=batch_size,#批量处理的数量
          epochs=epochs,#训练次数
          verbose=1,#行为区别
          validation_data=(x_test,y_test)) #验证的数据
score = model.evaluate(x_test,y_test,verbose=0) #评分
print("  ",score)

60000 样本数量
10000 样本数量
(60000, 28, 28, 1) (10000, 28, 28, 1)


e:\anaconda3\lib\site-packages\ipykernel_launcher.py:42: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="relu", input_shape=(28, 28, 1...)`
e:\anaconda3\lib\site-packages\ipykernel_launcher.py:43: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu")`


Instructions for updating:
keep_dims is deprecated, use keepdims instead
Train on 60000 samples, validate on 10000 samples
Epoch 1/12
60000/60000 [==============================] - 371s 6ms/step - loss: 0.3412 - acc: 0.8963 - val_loss: 0.0800 - val_acc: 0.9740
Epoch 2/12
60000/60000 [==============================] - 332s 6ms/step - loss: 0.1184 - acc: 0.9652 - val_loss: 0.0570 - val_acc: 0.9824
Epoch 3/12
60000/60000 [==============================] - 306s 5ms/step - loss: 0.0902 - acc: 0.9730 - val_loss: 0.0456 - val_acc: 0.9836
Epoch 4/12
60000/60000 [==============================] - 251s 4ms/step - loss: 0.0767 - acc: 0.9773 - val_loss: 0.0416 - val_acc: 0.9864
Epoch 5/12
60000/60000 [==============================] - 249s 4ms/step - loss: 0.0684 - acc: 0.9800 - val_loss: 0.0367 - val_acc: 0.9873
Epoch 6/12
60000/60000 [==============================] - 250s 4ms/step - loss: 0.0615 - acc: 0.9816 - val_loss: 0.0346 - val_acc: 0.9879
Epoch 7/12
60000/60000 [=========================

NameError: name 'socre' is not defined

In [15]:
print("  ",score)

   [0.028595277952877224, 0.9903]


In [10]:
# -*- coding: utf-8 -*-  
'''Trains a simple deep NN on the MNIST dataset.

Gets to 98.40% test accuracy after 20 epochs
(there is *a lot* of margin for parameter tuning).
2 seconds per epoch on a K520 GPU.
'''

from __future__ import print_function

import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop

batch_size = 128
num_classes = 10
epochs = 20

# the data, shuffled and split between train and test sets 
# (x_train, y_train), (x_test, y_test) = mnist.load_data()

import numpy as np
path='./MNIST_data/mnist.npz'
f = np.load(path)
x_train, y_train = f['x_train'], f['y_train']
x_test, y_test = f['x_test'], f['y_test']
f.close()

x_train = x_train.reshape(60000, 784).astype('float32')
x_test = x_test.reshape(10000, 784).astype('float32')
x_train /= 255
x_test /= 255
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
# label为0~9共10个类别，keras要求格式为binary class matrices

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

# add by hcq-20171106
# Dense of keras is full-connection.
model = Sequential()
model.add(Dense(512, activation='relu', input_shape=(784,)))
model.add(Dropout(0.2))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(num_classes, activation='softmax'))

model.summary()

model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

60000 train samples
10000 test samples
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 512)               401920    
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)               262656    
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                5130      
Total params: 669,706
Trainable params: 669,706
Non-trainable params: 0
_________________________________________________________________
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updatin